<a href="https://colab.research.google.com/github/arjunks25/CancerDetectionAI/blob/main/CancerDetectionAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import zipfile

with zipfile.ZipFile('/content/drive/My Drive/BrainMRIDataset.zip', 'r') as zip_ref:
    zip_ref.extractall('/content')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install --upgrade torchvision

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
import os

# Define data transformations, including data augmentation
transform = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

# Define data directories
data_dir = '/content'
train_dir = os.path.join(data_dir, 'Training')
test_dir = os.path.join(data_dir, 'Testing')
# Map class labels to cancer types
class_to_cancer_type = {
    0: "Glioma Tumor",
    1: "Meningioma Tumor",
    2: "No Tumor",
    3: "Pituitary Tumor"
}
# Create data loaders for the training and testing datasets
batch_size = 32

train_dataset = datasets.ImageFolder(train_dir, transform=transform['train'])
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = datasets.ImageFolder(test_dir, transform=transform['test'])
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Use a small pre-trained model (ResNet-18) from torchvision
model = models.resnet18(pretrained=True)

# Modify the classifier to match the number of classes (4 in your case)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 4)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Define a learning rate scheduler for better training
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

early_stopping = 1  # Number of epochs without improvement to wait before stopping
best_accuracy = 0.0  # Track the best accuracy
wait = 0  # Counter for early stopping

In [ ]:
# Training
num_epochs = 12

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()


    scheduler.step()  # Adjust learning rate

    print(f'Epoch {epoch + 1}, Loss: {running_loss / len(train_loader)}')

print('Finished Training')

Epoch 1, Loss: 0.41677393239803157
Epoch 2, Loss: 0.2156177366662292
Epoch 3, Loss: 0.20052473480142027
Epoch 4, Loss: 0.16395441136796382
Epoch 5, Loss: 0.14947952143847942
Epoch 6, Loss: 0.13716440175415417
Epoch 7, Loss: 0.1265314271226109
Epoch 8, Loss: 0.10231919449821697
Epoch 9, Loss: 0.09536494384389683
Epoch 10, Loss: 0.08558979674955237
Epoch 11, Loss: 0.08257844056101651
Epoch 12, Loss: 0.06973916896509641
Epoch 13, Loss: 0.08262787218460574
Epoch 14, Loss: 0.07676189948867111
Epoch 15, Loss: 0.07199795143262551
Epoch 16, Loss: 0.06806552323236978
Epoch 17, Loss: 0.0696693676986924
Epoch 18, Loss: 0.08347048468409138
Epoch 19, Loss: 0.07460449083172326
Epoch 20, Loss: 0.07678055826980391
Epoch 21, Loss: 0.06781260557368647
Epoch 22, Loss: 0.06526254781641525
Epoch 23, Loss: 0.06983190354521714
Epoch 24, Loss: 0.0686210528557194
Epoch 25, Loss: 0.06854056322416446
Finished Training


In [ ]:
# Testing and Calculating Accuracy
model.eval()
correct = [0, 0, 0, 0]
total = [0, 0, 0, 0]

with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        c = (predicted == labels).squeeze()
        for i in range(len(labels)):
             label = labels[i]
             correct[label] += c[i].item()
             total[label] += 1

overall_accuracy = 100 * sum(correct) / sum(total)

if overall_accuracy > best_accuracy:
     best_accuracy = overall_accuracy
     wait = 0
else:
    wait += 1

print(f'Overall Accuracy on the test set: {overall_accuracy:.2f}%')

for i in range(4):
    class_accuracy = 100 * correct[i] / total[i]
    cancer_type = class_to_cancer_type[i]
    print(f'Accuracy for {cancer_type}: {class_accuracy:.2f}%')


if wait >= early_stopping:
    print(f'Early stopping after {early_stopping} epochs without improvement.')


print('Finished Testing')

Overall Accuracy on the test set: 99.31%
Accuracy for Glioma Tumor: 97.67%
Accuracy for Meningioma Tumor: 100.00%
Accuracy for No Tumor: 100.00%
Accuracy for Pituitary Tumor: 99.33%
Finished Testing


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Saving the current model to Google Drive
torch.save(model.state_dict(), '/content/drive/My Drive/brain_tumor_model_resnet18.pth')